In [62]:
import pandas as pd
import numpy as np
import os
import re

%matplotlib inline

In [30]:
def print_memory_usage(df):
    print(df.memory_usage(index=True, deep=True).sum()/2**20, 'Mb')

In [58]:
def read_tosvmr(filename):
    if os.stat(filename).st_size == 0:
        return None
    df = pd.read_csv(filename, sep=' ', header=None)
    del df[7], df[6]
    df.columns = ['Rank', 'QueryID', 'CTR', 'CTP',
                  'Purchases', 'Turnover', 'ItemID']
    df.QueryID = df.QueryID.str.extract('qid:(\d+)').astype('int')
    for i, col in enumerate(['CTR', 'CTP', 'Purchases', 'Turnover']):
        df[col] = df[col].str.extract(str(i+1) + ':(.+)').astype('float')
    df.ItemID = df.ItemID.str.extract('item_id=(.+)')
    return df

In [63]:
directory = '../data/svmr/' 
for filename in [f for f in os.listdir(directory) if re.match('.+\.tosvmr', f)]:
    print(filename)

04EE603A-367E-43E6-97DF-59C39E890337.tosvmr
7B4C44E1-4579-485A-8F66-0908A527BC8D.tosvmr
5AED100E-A9BC-441B-A185-8BBB37B5D49E.tosvmr
DF8AE980-C37E-4D70-ABF1-B0B6D0C43E36.tosvmr
4158565D-E3D4-49FF-88CE-60D458C9D008.tosvmr
59879A85-C14C-4CE4-96E8-3D2C33FAFEA7.tosvmr
ABC129E4-84B9-44C7-BE32-AFB5E4DFC194.tosvmr
801348B2-5F3A-4BE5-AAEF-41D52521397F.tosvmr
3A4B77E0-9B49-43C4-9EF0-C17C3F7124F0.tosvmr
06A40236-B6BD-441C-800C-6F6EE2523787.tosvmr
BAB91AE9-E41E-4363-8F5C-62A1B7CC0F76.tosvmr
7EC8358B-3FA1-45FC-8474-E8C9A621C253.tosvmr
9A97FD42-A54B-4B80-95AF-9D09ACE75B81.tosvmr
30F88499-4445-45CC-BD75-5E73EE272ABE.tosvmr
A1060FF6-374C-4E28-BAAF-95355939CEB0.tosvmr
F5EA4E13-5AC3-4A31-9E94-C5A1378F02FA.tosvmr
D49D68F9-12BD-42BC-A979-8ACC572F3367.tosvmr
4E090B1B-D1A6-469C-8A06-50B735EE8687.tosvmr
F173748B-A335-401F-B233-A35FFB7E5A7A.tosvmr
D1DAB1F0-ECED-450F-B653-5502DEFFB292.tosvmr
A399C2F7-3337-4A6A-B969-95D0CBC1245E.tosvmr
CCDE62D3-4DB6-4492-96B4-44488F85B49A.tosvmr
8D778768-7723-44BB-8A00-4956EA9D